In [1]:
%run _standard_imports.ipynb

python 3.4.3 |Anaconda 2.2.0 (64-bit)| (default, Mar  6 2015, 12:03:53) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.9.2
scipy 0.15.1
pandas 0.15.2
numexpr 2.3.1
pysam 0.8.3
pysamstats 0.23
petl 1.0.11
petlx 1.0.3
vcf 0.6.7
vcfnp 2.3.0.dev0
h5py 2.4.0
tables 3.1.1


In [2]:
pf_40_freeze_manifest_fn = '/nfs/team112_internal/production_files/Pf/4_0/pf_40_freeze_manifest.tab'
samples_subset_fn = '/nfs/team112_internal/rp7/data/Pf/4_0/meta/PfCP_4.txt' # From Roberto March 30, 2106 14:14
output_dir = '/lustre/scratch109/malaria/rp7/data/Pf/4_0/pysamstats'

analysis_chroms = collections.OrderedDict()
analysis_chroms['Pf3D7_05_v3'] = [37901, 1321390]
analysis_chroms['Pf3D7_08_v3'] = [1, 1472805]
analysis_chroms['Pf3D7_09_v3'] = [79101, 1473560]
analysis_chroms['Pf3D7_13_v3'] = [1, 2925236]


In [3]:
tbl_samples_subset = etl.fromtsv(samples_subset_fn).pushheader(['sample']).addfield('temp', 0)
print(len(tbl_samples_subset.data()))
tbl_samples_subset

3488


sample,temp
PA0007-C,0
PA0008-C,0
PA0011-C,0
PA0012-C,0
PA0015-C,0


In [4]:
tbl_pf_40_freeze_manifest = (etl
    .fromtsv(pf_40_freeze_manifest_fn)
    .sub('path', 'scratch108', 'scratch109')
    .join(tbl_samples_subset, key='sample')
)
print(len(tbl_pf_40_freeze_manifest.data()))
tbl_pf_40_freeze_manifest.display(index_header=True)

3488


0|path,1|study,2|sample,3|bases,4|bases_mapped,5|avg_read_length,6|bases_of_1X_coverage,7|bases_of_5X_coverage,8|bases_of_10X_coverage,9|bases_of_50X_coverage,10|mean_coverage,11|sample_acc,12|run_acc,13|temp
/lustre/scratch109/malaria/pfalciparum/output/2/d/d/a/42721/1_bam_merge/pe.1.bam,PF6,PA0007-C,2074610304,1849037212,76,22703916,20226914,17341168,9063212,74.27,ERS010116,ERR018933,0
/lustre/scratch109/malaria/pfalciparum/output/0/2/f/9/42722/1_bam_merge/pe.1.bam,PF6,PA0008-C,2529943176,2302964844,76,23160225,22873797,22241258,14633143,96.81,ERS010126,ERR018936,0
/lustre/scratch109/malaria/pfalciparum/output/3/d/9/5/42723/1_bam_merge/pe.1.bam,PF6,PA0011-C,3707830088,3451499872,76,22960209,22798857,22697202,21815652,149.59,ERS010043,ERR015410,0
/lustre/scratch109/malaria/pfalciparum/output/5/6/7/1/42724/1_bam_merge/pe.1.bam,PF6,PA0012-C,2910823712,2219976340,76,22982697,22430685,21499677,14970548,95.02,ERS010127,ERR021977,0
/lustre/scratch109/malaria/pfalciparum/output/b/a/0/5/42725/1_bam_merge/pe.1.bam,PF6,PA0013-C,1706925600,1536128300,100,22819992,22447606,21882221,14505222,65.28,ERS157495,ERR211480,0


In [33]:
for chrom in 
for i, rec in enumerate(tbl_pf_40_freeze_manifest.head(3).data()):
    bam_fn = rec[0]
    ox_code = rec[2].replace('-', '_')
    print(i, ox_code)
#     print("\n", ox_code, end=":")
#     for chrom in analysis_chroms:
    for chrom in ['Pf3D7_05_v3']:
#         print(chrom, end=", ")
        output_fn = "%s/results/%s/%s_%s.coverage_300bp.txt" % (output_dir, chrom, ox_code, chrom)
        print(output_fn)

0 PA0007_C
/lustre/scratch109/malaria/rp7/data/Pf/4_0/pysamstats/results/Pf3D7_05_v3/PA0007_C_Pf3D7_05_v3.coverage_300bp.txt
1 PA0008_C
/lustre/scratch109/malaria/rp7/data/Pf/4_0/pysamstats/results/Pf3D7_05_v3/PA0008_C_Pf3D7_05_v3.coverage_300bp.txt
2 PA0011_C
/lustre/scratch109/malaria/rp7/data/Pf/4_0/pysamstats/results/Pf3D7_05_v3/PA0011_C_Pf3D7_05_v3.coverage_300bp.txt


In [39]:
number_of_samples = len(tbl_pf_40_freeze_manifest.data())

loc_arrays = collections.OrderedDict()
pp_arrays = collections.OrderedDict()

for chrom in analysis_chroms:
    print(chrom)
    loc_arrays[chrom] = (
        etl
        .fromtsv(
            '/lustre/scratch109/malaria/rp7/data/Pf/4_0/pysamstats/results/%s/PA0007_C_%s.coverage_300bp.txt' % (
            chrom, chrom)
        )
        .convertnumbers()
        .toarray()
    )
    number_of_bins = loc_arrays[chrom].shape[0]
    pp_arrays[chrom] = np.zeros((number_of_bins, number_of_samples), dtype='i2')
#     pp_arrays[chrom] = np.zeros((number_of_bins, 100), dtype='i2')
    for i, rec in enumerate(tbl_pf_40_freeze_manifest.data()):
#     for i, rec in enumerate(tbl_pf_40_freeze_manifest.head(100).data()):
        ox_code = rec[2].replace('-', '_')
        print('.', end='')
#         print(i, ox_code)
        output_fn = "%s/results/%s/%s_%s.coverage_300bp.txt" % (output_dir, chrom, ox_code, chrom)
        pp_arrays[chrom][:,i] = (
            etl
            .fromtsv(output_fn)
            .convertnumbers()
            .values('reads_pp')
            .array()
        )

        
    

Pf3D7_05_v3
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [41]:
import pickle

pp_arrays_fn = "%s/pp_arrays.p" % output_dir
loc_arrays_fn = "%s/loc_arrays.p" % output_dir
pickle.dump(pp_arrays, open(pp_arrays_fn, "wb"))
pickle.dump(loc_arrays, open(loc_arrays_fn, "wb"))


In [44]:
tbl_pf_40_freeze_manifest.selecteq('study', 'PF13').displayall()

path,study,sample,bases,bases_mapped,avg_read_length,bases_of_1X_coverage,bases_of_5X_coverage,bases_of_10X_coverage,bases_of_50X_coverage,mean_coverage,sample_acc,run_acc,temp
/lustre/scratch109/malaria/pfalciparum/output/6/8/e/6/45077/1_bam_merge/pe.1.bam,PF13,PP0002-C,2130705000,1784484378,54,22002598,18587475,16053066,9364981,71.40,ERS010192,"ERR012671,ERR012834,ERR037220,ERR038339",0
/lustre/scratch109/malaria/pfalciparum/output/e/a/6/f/45079/1_bam_merge/pe.1.bam,PF13,PP0004-C,1850171976,1648178658,54,22210489,19320379,17009863,9727720,67.87,ERS010215,"ERR012297,ERR012313",0
/lustre/scratch109/malaria/pfalciparum/output/c/7/b/5/45080/1_bam_merge/pe.1.bam,PF13,PP0005-C,5180304856,4876140164,76,23020182,22762104,22615477,21012784,205.22,ERS010050,ERR019545,0
/lustre/scratch109/malaria/pfalciparum/output/b/0/f/0/45084/1_bam_merge/pe.1.bam,PF13,PP0010-C,5491376656,5185208300,76,22900138,22736019,22634619,22109154,225.99,ERS011443,ERR022855,0
/lustre/scratch109/malaria/pfalciparum/output/d/5/b/b/45085/1_bam_merge/pe.1.bam,PF13,PP0011-C,3948925648,3706452284,76,22894587,22572644,22413108,21657540,161.84,ERS011444,ERR022856,0
/lustre/scratch109/malaria/pfalciparum/output/3/e/8/e/45086/1_bam_merge/pe.1.bam,PF13,PP0012-C,2301168888,2132593060,76,22711904,22068954,21299672,16563957,92.16,ERS016330,ERR027113,0
/lustre/scratch109/malaria/pfalciparum/output/1/b/a/5/45087/1_bam_merge/pe.1.bam,PF13,PP0013-C,1578132400,1471258500,100,22695529,22390972,22036948,15631751,64.16,ERS071916,ERR123900,0
/lustre/scratch109/malaria/pfalciparum/output/9/3/f/0/45088/1_bam_merge/pe.1.bam,PF13,PP0014-C,1745090000,1592566800,100,22726525,22365573,21840018,13741343,68.91,ERS071910,ERR123901,0
/lustre/scratch109/malaria/pfalciparum/output/2/7/2/8/45089/1_bam_merge/pe.1.bam,PF13,PP0015-C,1755785200,1144014500,100,22706364,22384403,21980246,11116792,49.55,ERS071911,ERR123902,0
/lustre/scratch109/malaria/pfalciparum/output/f/e/7/8/45090/1_bam_merge/pe.1.bam,PF13,PP0016-C,1837299200,1622456700,100,22724040,22328140,21704851,14467763,70.39,ERS071917,ERR123903,0


In [43]:
!samtools view /lustre/scratch109/malaria/pfalciparum/output/6/8/e/6/45077/1_bam_merge/pe.1.bam \
| grep -E 'ATGATGGGCAGGGTTCAGGGTTTA' | wc -l

130


In [45]:
!samtools view /lustre/scratch109/malaria/pfalciparum/output/e/a/6/f/45079/1_bam_merge/pe.1.bam \
| grep -E 'ATGATGGGCAGGGTTCAGGGTTTA' | wc -l

0


In [57]:
import stat
bsub = sh.Command('bsub')

# Actually the breakpoint is at 1374933.
breakpoint_reads_dir = '/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_08_v3_1374895'
!mkdir -p {breakpoint_reads_dir}/results
!mkdir -p {breakpoint_reads_dir}/scripts
!mkdir -p {breakpoint_reads_dir}/logs

breakpoint_sequence = "'ATGATGGGCAGGGTTCAGGGTTTA'"
# breakpoint_spanning_reads_fn = "%s/looser_breakpoint_spanning_reads.p" % plot_dir
for i, rec in enumerate((etl
    .fromtsv(pf_40_freeze_manifest_fn)
    .sub('path', 'scratch108', 'scratch109')
).data()):
# for rec in tbl_all_1_0_samples.data():
#     print(rec[0])
    bam_fn = rec[0]
    ox_code = rec[2].replace('-', '_')
    print('.', end='')
    num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
    if not os.path.exists(num_breakpoint_reads_fn):
        script_fn = "%s/scripts/nbpr_%s.sh" % (breakpoint_reads_dir, ox_code)
        fo = open(script_fn, 'w')
#         print(script_fn)
        print('samtools view %s | grep -E %s | wc -l > %s' % (bam_fn, breakpoint_sequence, num_breakpoint_reads_fn), file = fo)
        fo.close()
        st = os.stat(script_fn)
        os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
        bsub(
            '-G', 'malaria-dk',
            '-P', 'malaria-dk',
            '-q', 'normal',
            '-o', '%s/logs/nbpr_%s.out' % (breakpoint_reads_dir, ox_code),
            '-e', '%s/logs/nbpr_%s.err' % (breakpoint_reads_dir, ox_code),
            '-J', 'nbpr_%s' % (ox_code),
            '-R', "'select[mem>4000] rusage[mem=4000]'",
            '-M', '4000',
            script_fn
        )


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [63]:
!egrep -v '^0$' /lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_08_v3_1374895/results/*

/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_08_v3_1374895/results/num_breakpoint_reads_PP0002_C.txt:130
/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_08_v3_1374895/results/num_breakpoint_reads_PP0010_C.txt:18
/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_08_v3_1374895/results/num_breakpoint_reads_PP0017_C.txt:54


In [59]:
import stat
bsub = sh.Command('bsub')

breakpoint_reads_dir = '/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_08_v3_1366025'
!mkdir -p {breakpoint_reads_dir}/results
!mkdir -p {breakpoint_reads_dir}/scripts
!mkdir -p {breakpoint_reads_dir}/logs

# breakpoint_sequence = "'ATGATGGGCAGGGTTCAGGGTTTA'"
breakpoint_sequence = "'CTGGTGGGATTTTCAGGGTTTAGGG'"

# breakpoint_spanning_reads_fn = "%s/looser_breakpoint_spanning_reads.p" % plot_dir
for i, rec in enumerate((etl
    .fromtsv(pf_40_freeze_manifest_fn)
    .sub('path', 'scratch108', 'scratch109')
).data()):
# for rec in tbl_all_1_0_samples.data():
#     print(rec[0])
    bam_fn = rec[0]
    ox_code = rec[2].replace('-', '_')
    print('.', end='')
    num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
    if not os.path.exists(num_breakpoint_reads_fn):
        script_fn = "%s/scripts/nbpr_%s.sh" % (breakpoint_reads_dir, ox_code)
        fo = open(script_fn, 'w')
#         print(script_fn)
        print('samtools view %s | grep -E %s | wc -l > %s' % (bam_fn, breakpoint_sequence, num_breakpoint_reads_fn), file = fo)
        fo.close()
        st = os.stat(script_fn)
        os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
        bsub(
            '-G', 'malaria-dk',
            '-P', 'malaria-dk',
            '-q', 'normal',
            '-o', '%s/logs/nbpr_%s.out' % (breakpoint_reads_dir, ox_code),
            '-e', '%s/logs/nbpr_%s.err' % (breakpoint_reads_dir, ox_code),
            '-J', 'nbpr_%s' % (ox_code),
            '-R', "'select[mem>4000] rusage[mem=4000]'",
            '-M', '4000',
            script_fn
        )


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [62]:
!egrep -v '^0$' /lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_08_v3_1366025/results/* | grep -v PG

/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_08_v3_1366025/results/num_breakpoint_reads_PA0106_CW.txt:1
/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_08_v3_1366025/results/num_breakpoint_reads_PC0080_C.txt:141
/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_08_v3_1366025/results/num_breakpoint_reads_PC0083_C.txt:185
/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_08_v3_1366025/results/num_breakpoint_reads_PH0029_Cx.txt:361
/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_08_v3_1366025/results/num_breakpoint_reads_PH0562_C.txt:1
/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_08_v3_1366025/results/num_breakpoint_reads_PK0038_C.txt:5
/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_08_v3_1366025/results/num_breakpoint_reads_PM0017_C.txt:1
/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spannin

The above analysis showed the following samples have same deletion breakpoint as Dd2:
- PC0080_C (PF15, Kenya, in vitro drug resistance assay)
- PC0083_C (PF15, Kenya, in vitro drug resistance assay)
- PH0029_Cx (PF11, Cambodia, field sample? Olivo 16/06/2015 11:42 identified this as possible sample mislabelling)

This one has a different breakpoint
- PN0008_C (PF21, PNG, Pf lab samples. Appears deleted from PF3D7_0832200, PHISTa)

Also the following dubious matches
- PK0038_C (PF4, Bukina Faso, 5 matching reads. Maybe contaminated with Dd2?)
- PM0017_C (PF1, Mali, 1 matching read. Maybe a sequencing error?)
- PH0562_C (TRAC1, Cambodia. 1 matching read. Appears deleted from PF3D7_0832100, rifin pseudogene)
- PA0106_CW (PFlab7, Gambia. 1 matching read)


In [60]:
(etl.fromtsv(pf_40_freeze_manifest_fn).selectin('sample', ['PC0080-C', 'PC0083-C']))

path,study,sample,bases,bases_mapped,avg_read_length,bases_of_1X_coverage,bases_of_5X_coverage,bases_of_10X_coverage,bases_of_50X_coverage,mean_coverage,sample_acc,run_acc
/lustre/scratch108/malaria/pfalciparum/output/4/d/8/3/43032/1_bam_merge/pe.1.bam,PF15,PC0080-C,4499867736,4208217660,76,22440287,21446471,20675365,17347882,185.08,ERS017462,ERR029415
/lustre/scratch108/malaria/pfalciparum/output/1/5/b/9/43033/1_bam_merge/pe.1.bam,PF15,PC0083-C,4317727048,4081382704,76,22282962,21041274,20225426,16856031,180.73,ERS017463,ERR029416


In [61]:
for rec in (etl.fromtsv(pf_40_freeze_manifest_fn).selectin('sample', ['PC0080-C', 'PC0083-C'])).data():
    print('"%s" : { "bam":"%s" , "species" : "pf_3d7_v3" , "alg" : "bwa" , "group" : "%s" } ,' % (
            rec[2].replace('-', '_'),
            rec[0],
            rec[1]
            ))


"PC0080_C" : { "bam":"/lustre/scratch108/malaria/pfalciparum/output/4/d/8/3/43032/1_bam_merge/pe.1.bam" , "species" : "pf_3d7_v3" , "alg" : "bwa" , "group" : "PF15" } ,
"PC0083_C" : { "bam":"/lustre/scratch108/malaria/pfalciparum/output/1/5/b/9/43033/1_bam_merge/pe.1.bam" , "species" : "pf_3d7_v3" , "alg" : "bwa" , "group" : "PF15" } ,


In [55]:
(etl
    .fromtsv(pf_40_freeze_manifest_fn)
    .sub('path', 'scratch108', 'scratch109')
).selectin('study', ['1104-PF-LAB-WENDLER', 'PFprog1', 'PFlab2', 'PFlab3']).displayall()

path,study,sample,bases,bases_mapped,avg_read_length,bases_of_1X_coverage,bases_of_5X_coverage,bases_of_10X_coverage,bases_of_50X_coverage,mean_coverage,sample_acc,run_acc
/lustre/scratch109/malaria/pfalciparum/output/e/b/8/6/144292/1_bam_merge/pe.1.bam,1104-PF-LAB-WENDLER,PG0049-CW2,2736067200,2581541400,100,22904182,22678473,22472611,19844359,112.69,ERS319118,ERR369774
/lustre/scratch109/malaria/pfalciparum/output/a/3/3/8/144293/1_bam_merge/pe.1.bam,1104-PF-LAB-WENDLER,PG0389-C,3140646200,2956391900,100,23316893,23278289,23228987,21516269,126.39,ERS319116,ERR369760
/lustre/scratch109/malaria/pfalciparum/output/0/c/0/2/144294/1_bam_merge/pe.1.bam,1104-PF-LAB-WENDLER,PG0390-C,3422374400,3225309500,100,23318577,23259201,23156696,21175828,137.89,ERS319119,ERR369762
/lustre/scratch109/malaria/pfalciparum/output/e/8/f/8/144295/1_bam_merge/pe.1.bam,1104-PF-LAB-WENDLER,PG0391-C,3364975600,3172359000,100,23321086,23282984,23212228,21387876,135.63,ERS319122,ERR369764
/lustre/scratch109/malaria/pfalciparum/output/c/c/4/e/144296/1_bam_merge/pe.1.bam,1104-PF-LAB-WENDLER,PG0392-C,3248522800,3068777200,100,23322729,23296463,23239624,21128860,131.23,ERS319125,ERR369765
/lustre/scratch109/malaria/pfalciparum/output/b/a/b/e/144297/1_bam_merge/pe.1.bam,1104-PF-LAB-WENDLER,PG0393-C,3041002000,2843944100,100,23316321,23255578,23121623,19674284,121.66,ERS319128,ERR369766
/lustre/scratch109/malaria/pfalciparum/output/8/b/3/8/144298/1_bam_merge/pe.1.bam,1104-PF-LAB-WENDLER,PG0394-C,3172401400,2999408300,100,23321213,23264063,23088715,20021002,128.26,ERS319130,ERR369767
/lustre/scratch109/malaria/pfalciparum/output/5/0/f/1/144299/1_bam_merge/pe.1.bam,1104-PF-LAB-WENDLER,PG0395-C,3022613800,2843690400,100,23030382,22830817,22648289,19764013,123.18,ERS319132,ERR369768
/lustre/scratch109/malaria/pfalciparum/output/2/4/8/1/144300/1_bam_merge/pe.1.bam,1104-PF-LAB-WENDLER,PG0396-C,2726099800,2500953200,100,23039796,22791944,22522632,18531472,108.31,ERS319134,ERR369769
/lustre/scratch109/malaria/pfalciparum/output/8/0/c/d/144301/1_bam_merge/pe.1.bam,1104-PF-LAB-WENDLER,PG0397-C,2884785200,2715387000,100,22983795,22744165,22492010,19116898,117.90,ERS319136,ERR369770


In [56]:
for rec in (etl
    .fromtsv(pf_40_freeze_manifest_fn)
    .sub('path', 'scratch108', 'scratch109')
).selectin('study', ['1104-PF-LAB-WENDLER', 'PFprog1', 'PFlab2', 'PFlab3']).data():
    print('"%s" : { "bam":"%s" , "species" : "pf_3d7_v3" , "alg" : "bwa" , "group" : "%s" } ,' % (
            rec[2].replace('-', '_'),
            rec[0],
            rec[1]
            ))


"PG0049_CW2" : { "bam":"/lustre/scratch109/malaria/pfalciparum/output/e/b/8/6/144292/1_bam_merge/pe.1.bam" , "species" : "pf_3d7_v3" , "alg" : "bwa" , "group" : "1104-PF-LAB-WENDLER" } ,
"PG0389_C" : { "bam":"/lustre/scratch109/malaria/pfalciparum/output/a/3/3/8/144293/1_bam_merge/pe.1.bam" , "species" : "pf_3d7_v3" , "alg" : "bwa" , "group" : "1104-PF-LAB-WENDLER" } ,
"PG0390_C" : { "bam":"/lustre/scratch109/malaria/pfalciparum/output/0/c/0/2/144294/1_bam_merge/pe.1.bam" , "species" : "pf_3d7_v3" , "alg" : "bwa" , "group" : "1104-PF-LAB-WENDLER" } ,
"PG0391_C" : { "bam":"/lustre/scratch109/malaria/pfalciparum/output/e/8/f/8/144295/1_bam_merge/pe.1.bam" , "species" : "pf_3d7_v3" , "alg" : "bwa" , "group" : "1104-PF-LAB-WENDLER" } ,
"PG0392_C" : { "bam":"/lustre/scratch109/malaria/pfalciparum/output/c/c/4/e/144296/1_bam_merge/pe.1.bam" , "species" : "pf_3d7_v3" , "alg" : "bwa" , "group" : "1104-PF-LAB-WENDLER" } ,
"PG0393_C" : { "bam":"/lustre/scratch109/malaria/pfalciparum/output/b/a/b

In [ ]:
Dd2_ox_codes = ['PG0012-Cx', 'PG0008-C', 'PG0008-CW']

In [37]:
pp_arrays['Pf3D7_05_v3']

array([[ 93, 327, 819],
       [ 58, 193, 588],
       [ 64, 226, 697],
       ..., 
       [ 73, 151, 724],
       [ 15,  81, 545],
       [  2,  19, 121]], dtype=int16)

In [25]:
array_PA0007_C_Pf3D7_05_v3 = (
    etl
    .fromtsv('/lustre/scratch109/malaria/rp7/data/Pf/4_0/pysamstats/results/Pf3D7_05_v3/PA0007_C_Pf3D7_05_v3.coverage_300bp.txt')
    .convertnumbers()
    .toarray()
)

In [27]:
pp_PA0007_C_Pf3D7_05_v3 = (
    etl
    .fromtsv('/lustre/scratch109/malaria/rp7/data/Pf/4_0/pysamstats/results/Pf3D7_05_v3/PA0007_C_Pf3D7_05_v3.coverage_300bp.txt')
    .convertnumbers()
    .values('reads_pp')
    .array()
)

In [32]:
pp_PA0007_C_Pf3D7_05_v3.shape

(4279,)

In [16]:
array_PA0007_C_Pf3D7_05_v3.shape

(4279,)

In [17]:
array_PA0007_C_Pf3D7_05_v3

array([('Pf3D7_05_v3', 38051, 17, 95, 93),
       ('Pf3D7_05_v3', 38351, 12, 58, 58),
       ('Pf3D7_05_v3', 38651, 15, 64, 64), ...,
       ('Pf3D7_05_v3', 1320851, 16, 73, 73),
       ('Pf3D7_05_v3', 1321151, 13, 15, 15),
       ('Pf3D7_05_v3', 1321451, 10, 2, 2)], 
      dtype=[('chrom', '<U11'), ('pos', '<i8'), ('gc', '<i8'), ('reads_all', '<i8'), ('reads_pp', '<i8')])

In [21]:
array_pp_Pf3D7_05_v3 = np.zeros((4279, 3488), dtype='i2')

In [24]:
array_pp_Pf3D7_05_v3.shape

(4279, 3488)

In [23]:
array_pp_Pf3D7_05_v3[0,0]

0